In [9]:
import torch
from torch import nn

# 指定cpu, gpu设备
torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')
# cpu, gpu0, gpu1

# 查询可用gpu数量
torch.cuda.device_count()


0

In [11]:
"""定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码"""


"""如果存在，则返回gpu(i)，否则返回cpu()"""
def try_gpu(i=0):
# 不输入参数则默认i = 0 
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
     # 如果当前可用gpu的总数大于等于i+1，则返回第i个gpu（从0计数）		
    return torch.device('cpu')
	# 否则证明当前没有更多可用gpu，则返回cpu


"""返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    # 所有可用gpu设备序号组成的列表devices
    
    return devices if devices else [torch.device('cpu')]
	# 如果列表devices不为空则证明此时有可用的gpu，则返回可用gpu序号列表；否则证明没有可用gpu，则返回cpu

try_gpu(), try_gpu(10), try_all_gpus()
# 测试函数功能
# try_gpu():检测是否有第i=0号gpu
# try_gpu(10):检测是否有第i=10号gpu
# try_all_gpus():返回所有可用gpu序号列表，如果没有gpu则返回cpu


(device(type='cpu'), device(type='cpu'), [device(type='cpu')])

In [12]:
"""我们可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。"""
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

In [13]:
"""我们在第一个gpu上创建张量变量X""" 
X = torch.ones(2, 3, device=try_gpu())
"""
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
"""

"""假设你至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量"""
Y = torch.rand(2, 3, device=try_gpu(1))
"""
tensor([[0.3432, 0.4088, 0.7725],
        [0.0571, 0.3341, 0.2544]], device='cuda:1')
"""

"\ntensor([[0.3432, 0.4088, 0.7725],\n        [0.0571, 0.3341, 0.2544]], device='cuda:1')\n"

In [14]:
"""将gpu(0)中的X复制到gpu(1)中的Z"""
Z = X.cuda(1)
print(X)
print(Z)

"""
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')
"""


"""现在数据在同一个GPU上（Z和Y都在），我们可以将它们相加。"""
Y + Z

"""
tensor([[1.3432, 1.4088, 1.7725],
        [1.0571, 1.3341, 1.2544]], device='cuda:1')
"""


"""如果变量Z已经存在于第i个GPU上，再调用Z.cuda(i)只会返回Z并不会复制并分配新内存"""
Z.cuda(1) is Z

"""
True
"""

AssertionError: Torch not compiled with CUDA enabled

In [15]:
"""
类似地，神经网络模型可以指定设备。 下面的代码将模型参数放在GPU上。
"""

net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

In [16]:
net(X)
"""
tensor([[0.5037],
        [0.5037]], device='cuda:0', grad_fn=<AddmmBackward>)
"""

net[0].weight.data.device
"""
device(type='cuda', index=0)
"""

"\ndevice(type='cuda', index=0)\n"